# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [16]:
import os
import numpy as np
import tensorflow as tf
import pickle 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils import np_utils

# TODO: Implement load the data here.
data_path=os.getcwd()+"/traffic-signs-data"
training_file = data_path+"/train2.p"
testing_file = data_path+"/test2.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

nb_classes=43

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
# TODO: Implement data normalization here.
X_train=(X_train.astype(np.float32)-127.5)/255
X_test=(X_test.astype(np.float32)-127.5)/255

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.4f to %.4f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [5]:
# TODO: Build a two-layer feedforward neural network with Keras here.
model=Sequential()
model.add(Dense(128, input_dim=32*32*3, activation='relu', name='hidden1'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax', name='output'))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [7]:
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print(Y_train.shape)
history=model.fit(X_train.reshape(-1, 32*32*3), Y_train, nb_epoch=2, batch_size=100)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

(29406, 43)
Epoch 1/2
29406/29406 [==============================] - 2s - loss: 1.0951 - acc: 0.6754     
Epoch 2/2
29406/29406 [==============================] - 2s - loss: 0.9540 - acc: 0.7180     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [25]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.
from sklearn.model_selection import train_test_split

data_path=os.getcwd()+"/traffic-signs-data"
training_file = data_path+"/train2.p"
testing_file = data_path+"/test2.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=(X_train.astype(np.float32)-127.5)/255
X_test=(X_test.astype(np.float32)-127.5)/255

X_train, X_val, y_train, y_val=train_test_split(
X_train, y_train, test_size=0.25, random_state=0)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train=X_train.reshape(-1,32*32*3)
X_val=X_val.reshape(-1,32*32*3)

In [26]:

history=model.fit(X_train, Y_train, nb_epoch=2, batch_size=100,
                 validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 2s - loss: 2.9286 - acc: 0.4850 - val_loss: 1.0801 - val_acc: 0.7503
Epoch 2/2
29406/29406 [==============================] - 2s - loss: 1.3674 - acc: 0.6550 - val_loss: 0.6926 - val_acc: 0.8486


**Validation Accuracy**: (fill in here)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [29]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
from keras.layers import Convolution2D, MaxPooling2D
data_path=os.getcwd()+"/traffic-signs-data"
training_file = data_path+"/train2.p"
testing_file = data_path+"/test2.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=(X_train.astype(np.float32)-127.5)/255
X_test=(X_test.astype(np.float32)-127.5)/255

X_train, X_val, y_train, y_val=train_test_split(
X_train, y_train, test_size=0.25, random_state=0)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

nb_filters=32
kernel_size=(3,3)

model=Sequential()
model.add(Convolution2D(nb_filters,kernel_size[0], kernel_size[1],
                       border_mode='valid', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 27s - loss: 1.2204 - acc: 0.6874 - val_loss: 0.5103 - val_acc: 0.8617
Epoch 2/2
29406/29406 [==============================] - 27s - loss: 0.2882 - acc: 0.9298 - val_loss: 0.2504 - val_acc: 0.9420


AssertionError: The validation accuracy is: 0.862

In [30]:

history=model.fit(X_train, Y_train, nb_epoch=2, batch_size=200,
                 validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 22s - loss: 0.1382 - acc: 0.9712 - val_loss: 0.1882 - val_acc: 0.9558
Epoch 2/2
29406/29406 [==============================] - 22s - loss: 0.1018 - acc: 0.9798 - val_loss: 0.1685 - val_acc: 0.9567


**Validation Accuracy**: (fill in here)

In [33]:
score = model.evaluate(X_test, Y_test, verbose=0)

# STOP: Do not change the tests below. Your implementation should pass these tests.
print('Test accuracy:%.4f'%score[1])

Test accuracy:0.8797


## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [35]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.
nb_filters=32
kernel_size=(3,3)
pool_size=(2,2)
model=Sequential()
model.add(Convolution2D(nb_filters,kernel_size[0], kernel_size[1],
                       border_mode='valid', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])



In [37]:
history=model.fit(X_train, Y_train, nb_epoch=2, batch_size=200,
                 validation_data=(X_val, Y_val))


Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 15s - loss: 0.3735 - acc: 0.9124 - val_loss: 0.5460 - val_acc: 0.8359
Epoch 2/2
29406/29406 [==============================] - 15s - loss: 0.2703 - acc: 0.9396 - val_loss: 0.2648 - val_acc: 0.9321


**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [68]:
import cv2
def process_image(img):
    # convert to YCrCrb
    yuv_img=cv2.cvtColor(img,cv2.COLOR_RGB2YCrCb)
    
    # equalise Y channel 
    yuv_img[:,:,0]=cv2.equalizeHist(yuv_img[:,:,0])
    
    return yuv_img

In [101]:
from keras.layers import Convolution2D, MaxPooling2D
data_path=os.getcwd()+"/traffic-signs-data"
training_file = data_path+"/train2.p"
testing_file = data_path+"/test2.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

nb_classes=43

In [102]:
hist,bin_edges=np.histogram(y_train, nb_classes)

# add extra samples to make the balance the distribution
extra_samples=(max(hist)-hist)
total_extra_samples=np.sum(extra_samples)
X_train_extra=np.empty((total_extra_samples,32,32,3),dtype=np.uint8)
y_train_extra=np.empty((total_extra_samples))
i=0
for cls in range(nb_classes):
    class_samples= X_train[y_train==cls]
    n=len(class_samples)
    for _ in range(extra_samples[cls]):
        original=class_samples[int(np.random.uniform()*n)]
        #copy=transform_image(original)
        copy=original
        X_train_extra[i]=np.reshape(copy,(1,32,32,3))
        y_train_extra[i]=np.array([cls])
        i+=1

X_train=np.vstack((X_train, X_train_extra))
y_train=np.hstack((y_train, np.array(y_train_extra)))

In [103]:
n_train_extra=len(X_train)
for i in range(n_train_extra):
    X_train[i]=process_image(X_train[i])

n_test=len(X_test)
for i in range(n_test):
    X_test[i]=process_image(X_test[i])    
    

In [104]:
#X_train=(X_train.astype(np.float32)-127.5)/255
#X_test=(X_test.astype(np.float32)-127.5)/255

X_train, X_val, y_train, y_val=train_test_split(
X_train, y_train, test_size=0.01, random_state=0)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)


In [107]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.
from keras.layers import BatchNormalization
from keras.optimizers import Adam
model=Sequential()

# Conv 1
nb_filters=64
kernel_size=(3,3)
pool_size=(2,2)
model.add(Convolution2D(nb_filters,kernel_size[0], kernel_size[1],
                       border_mode='valid', input_shape=(32,32,3)))
model.add(BatchNormalization(epsilon=1e-9))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=pool_size))

# Conv 2
nb_filters=64
kernel_size=(3,3)
pool_size=(2,2)
model.add(Convolution2D(nb_filters,kernel_size[0], kernel_size[1],
                       border_mode='valid'))
model.add(BatchNormalization(epsilon=1e-9))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=pool_size))

# Conv 3
"""
nb_filters=64
kernel_size=(3,3)
pool_size=(2,2)
model.add(Convolution2D(nb_filters,kernel_size[0], kernel_size[1],
                       border_mode='valid'))
model.add(BatchNormalization(epsilon=1e-9))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(MaxPooling2D(pool_size=pool_size))
"""

# Dense 1
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))

# Output
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

adam=Adam(lr=5e-3, decay=0.5)
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])



In [108]:
history=model.fit(X_train, Y_train, nb_epoch=10, batch_size=200,
                 validation_data=(X_val, Y_val))

Train on 95782 samples, validate on 968 samples
Epoch 1/10
70200/95782 [====================>.........] - ETA: 98s - loss: 1.0613 - acc: 0.6948

KeyboardInterrupt: 

**Validation Accuracy**: (fill in here)

In [109]:
score = model.evaluate(X_test, Y_test, verbose=0, batch_size=200)

# STOP: Do not change the tests below. Your implementation should pass these tests.
print('Test accuracy:%.4f'%score[1])

Test accuracy:0.8083


## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.